In [1]:
# import cell
import json
import numpy as np
import torch
from datasets import load_dataset
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import k_means
from clustering import Pythia_model
import pickle
import joblib

/home/sv-goat/dictionary_learning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Read the JSON file, get the key terms into correct a list
data = json.loads(open('mesh_terms.json').read())

In [3]:
parents = list(data.keys()) #data.keys()
print(parents)

['Anatomy', 'Diseases', 'Chemicals and Drugs', 'Phenomena and Processes']


In [4]:
# Get the children of each parent
children = []
for parent in parents:
    children.append(list(data[parent].keys()))
print(children)

[['Extremities', 'Cartilage', 'Liver', 'Lung'], ['Infections', 'Cysts', 'Arm Injuries', 'Drowning'], ['Organic Chemicals', 'Micelles', 'Carbohydrates', 'Biological Factors'], ['Physical Phenomena', 'Metabolism', 'Algorithms', 'Fourier Analysis']]


In [5]:
all_terms = []
for parent in parents:
    all_terms.append(parent)
for child in children:
    all_terms.extend(child)
print(all_terms)

['Anatomy', 'Diseases', 'Chemicals and Drugs', 'Phenomena and Processes', 'Extremities', 'Cartilage', 'Liver', 'Lung', 'Infections', 'Cysts', 'Arm Injuries', 'Drowning', 'Organic Chemicals', 'Micelles', 'Carbohydrates', 'Biological Factors', 'Physical Phenomena', 'Metabolism', 'Algorithms', 'Fourier Analysis']


In [6]:
### USE IF YOU WANNA INITIALIZE A DATASET WITH DIFFERENT PARAMETERS

# # Extract sentences from the Pile dataset that have these terms.
# pile_dataset = load_dataset("monology/pile-uncopyrighted", split="train", streaming=True)
# sentences_per_term = [0 for i in range(len(all_terms))]
# num_sentences = 0
# biomedical_sentences = [[] for i in range(len(all_terms))]
# for sentence in pile_dataset:
#     for i, term in enumerate(all_terms):
#         # Convert to lower case
#         term = term.lower()
#         if term in sentence["text"] and sentences_per_term[i] < 60:
#             sentences_per_term[i] += 1
#             biomedical_sentences[i].append(sentence["text"])
#             num_sentences += 1
#             break
#     if num_sentences >= 1000:
#         break

In [7]:
### UTIL BLOCK IF WE NEED TO DO IT FROM SCRATCH


# # Convert sentences into a dataframe
# df = pd.DataFrame()
# df['Term'] = []
# df['Sentence'] = []
# count = 0
# for i, term in enumerate(all_terms):
#     for sentence in biomedical_sentences[i]:
#         df.loc[count] =  [term, sentence]
#         count += 1
# # Save it
# df.to_csv("biomedical_sentences.csv")
#


In [8]:
# Load the dataframe
df = pd.read_csv("biomedical_sentences.csv")

In [9]:
# Load the Pythia model and the tokenizer
model = Pythia_model("EleutherAI/pythia-70m-deduped", "EleutherAI/pythia-70m-deduped")

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [10]:
# activations_of_interest = []
# for i in range(len(df)):
#     activations_of_interest.append(model.get_activation_for_token(df.iloc[i]['Sentence'], df.iloc[i]['Term']))

In [11]:
# with open('final_layer_activations.pkl', 'wb') as f:
#     pickle.dump(activations_of_interest, f)

In [12]:
# # Load the final layer activations
# with open('final_layer_activations.pkl', 'rb') as f:
#     activations_of_interest = pickle.load(f) #final_layer_activations

In [13]:
# # Get the activations without any None values
# non_none_activations = [i for i in activations_of_interest if i is not None]

In [14]:
# # None values are handled implicitly
# k_means = k_means.k_means(non_none_activations, 4, distance_metric = "Euclidean")
# # Save the k_means model
# # k_means.save("mesh_k_means")

In [15]:
# # SAve the k_means model
# joblib.dump(k_means, "mesh_k_means")

In [16]:
# # Load the k_means model
# k_means = joblib.load("mesh_k_means")

In [17]:
# preds = k_means.predict(non_none_activations)

In [18]:
# print(preds)

In [19]:
# # Visualize and validate the hypothesis.
# # Find out which activation belongs to which cluster and keyword.
# # Create a list of size all_terms
# common_keywords = [[] for i in range(len(all_terms))]
# count = 0
# for i in range(len(activations_of_interest)):
#     if activations_of_interest[i] is not None:
#         # Find whihch all term it belongs to
#         for j in range(len(all_terms)):
#             if all_terms[j].lower() in df.iloc[i]['Sentence'].lower():
#                 common_keywords[j].append(preds[count])
#                 count += 1
#                 break


In [20]:
# print(common_keywords[6])
# print(all_terms[6])

In [21]:
# # Find the cluster wise means, and print them. I wanna see something.
# for i in range(len(common_keywords)):
#     print(all_terms[i], np.mean(common_keywords[i]))

In [22]:
sparse_activations_of_interest = []
for i in range(len(df)):
     sparse_activations_of_interest.append(model.get_sparse_activations_for_token(df.iloc[i]['Sentence'], df.iloc[i]['Term']))

activations device cuda:0


AttributeError: 'AutoEncoder' object has no attribute 'device'

In [ ]:
# Save the sparse activations
with open('sparse_activations.pkl', 'wb') as f:
    pickle.dump(sparse_activations_of_interest, f)

I think this is a good starting point that we can iterate upon later.

Some things that I can think of:-
1. Add more sentences to the dataset.
2. Bigger models
3. Different distance metrics ( Cosine similarity / KL divergence )
4. Other clustering algorithms